In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import db_password

In [2]:
reviews = pd.read_csv("./Reviews.csv")
#print(type(reviews_df))
reviews_df = pd.DataFrame(reviews)

In [3]:
listings = pd.read_csv("./Listings.csv")
low_memory=False
listings_df = pd.DataFrame(listings)

C:\Users\Dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
reviews_df.head()

,listing_id,review_id,date,reviewer_id
0,11798,330265172,2018-09-30,11863072
1,15383,330103585,2018-09-30,39147453
2,16455,329985788,2018-09-30,1125378
3,17919,330016899,2018-09-30,172717984
4,26827,329995638,2018-09-30,17542859


In [5]:
listings_df.head()

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÂ² Paris (Sacre CÅ“ur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


## Create listing_info and listing_review_stats dataframes


In [6]:
#extract the data for the listing_info table from listings_df
listings_info = listings_df[["listing_id", "host_id", "name", "host_location", "minimum_nights",
                           "maximum_nights", "instant_bookable"]]
listings_info.head()

,listing_id,host_id,name,host_since,host_location,minimum_nights,maximum_nights,instant_bookable
0,281420,1466919,"Beautiful Flat in le Village Montmartre, Paris",2011-12-03,"Paris, Ile-de-France, France",2,1125,f
1,3705183,10328771,39 mÂ² Paris (Sacre CÅ“ur),2013-11-29,"Paris, Ile-de-France, France",2,1125,f
2,4082273,19252768,"Lovely apartment with Terrace, 60m2",2014-07-31,"Paris, Ile-de-France, France",2,1125,f
3,4797344,10668311,Cosy studio (close to Eiffel tower),2013-12-17,"Paris, Ile-de-France, France",2,1125,f
4,4823489,24837558,Close to Eiffel Tower - Beautiful flat : 2 rooms,2014-12-14,"Paris, Ile-de-France, France",2,1125,f


In [7]:
#extract the data for the listing_review_stats tables from listings_df, is still missing total reviews
listing_review_stats = listings_df[["listing_id", 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                                   'review_scores_communication','review_scores_location', 'review_scores_value']]
print(len(listing_review_stats))
listing_review_stats.head()

279712


,listing_id,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,281420,100.0,10.0,10.0,10.0,10.0,10.0,10.0
1,3705183,100.0,10.0,10.0,10.0,10.0,10.0,10.0
2,4082273,100.0,10.0,10.0,10.0,10.0,10.0,10.0
3,4797344,100.0,10.0,10.0,10.0,10.0,10.0,10.0
4,4823489,100.0,10.0,10.0,10.0,10.0,10.0,10.0


In [8]:
#get the total review count for each listing_id with a review and organize data
#there are listing_ids with no reviews, meaning the null values will need to be changed to 0 after merge
#with listing_review_stats unless there is no review information in the listings_id_df--look at this part of the data
listing_total_reviews = reviews_df['listing_id'].value_counts()
listing_total_reviews = pd.DataFrame(listing_total_reviews)
listing_total_reviews = listing_total_reviews.reset_index()
listing_total_reviews = listing_total_reviews.rename(columns={'listing_id':'total_reviews'})
listing_total_reviews = listing_total_reviews.rename(columns={'index':'listing_id'})
listing_total_reviews = pd.DataFrame(listing_total_reviews)
listing_total_reviews

,listing_id,total_reviews
0,17222007,891
1,8637229,828
2,1249964,796
3,32011332,762
4,2399029,754
...,...,...
193551,45914743,1
193552,41929967,1
193553,40561637,1
193554,36333146,1


In [9]:
#left join listing_total_reviews with listing_review_stats
listing_review_stats = pd.merge(listing_review_stats,listing_total_reviews,
                                how="left", on="listing_id")
listing_review_stats

,listing_id,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,total_reviews
0,281420,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
1,3705183,100.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0
2,4082273,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
3,4797344,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
4,4823489,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
...,...,...,...,...,...,...,...,...,...
279707,38338635,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
279708,38538692,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
279709,38683356,100.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0
279710,39659000,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0


In [10]:
print(listing_review_stats["review_scores_accuracy"].unique())

[10.  9.  8.  4.  7.  6.  5. nan  2.  3.]


In [11]:
#fill the null total_reviews with 0
listing_review_stats["total_reviews"].fillna(0, inplace= True)
#listing_review_stats['total_reviews'].unique()

#rename review columns to be specifically for listings
listing_review_stats = listing_review_stats.rename(columns={'review_scores_rating':'listing_rating', 'review_scores_accuracy':
                                                            'listing_score_accuracy', 'review_scores_checkin':'listing_score_checkin',
                                                          'review_scores_communication':'listing_score_communication', 
                                                            'review_scores_cleanliness': 'listing_score_cleanliness',
                                                            'review_scores_location':'listing_score_location','review_scores_value':'listing_score_value',
                                                           'total_reviews':'listing_total_reviews'})
#check to verify listing ids are unique (they are)
print(len(pd.unique(listing_review_stats['listing_id'])))
listing_review_stats.head()

279712


,listing_id,listing_rating,listing_score_accuracy,listing_score_cleanliness,listing_score_checkin,listing_score_communication,listing_score_location,listing_score_value,listing_total_reviews
0,281420,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
1,3705183,100.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0
2,4082273,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
3,4797344,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
4,4823489,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0


## Now move onto the host_review_stats dataframe

In [12]:
#modify the massive listings_df to include the total review data with
#the a left join so that it is easier to get total review data for 
#hosts and host locations dataframes
listings_df = pd.merge(listings_df, listing_total_reviews, how = 'left',
                      on='listing_id')
listings_df.head()

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,total_reviews
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,2.0
1,3705183,39 mÂ² Paris (Sacre CÅ“ur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,6.0
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,1.0
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,1.0
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,1.0


In [13]:
#fill the null total_reviews in listings_df with 0
listings_df["total_reviews"].fillna(0, inplace= True)

In [14]:
#begin the host_reviews_stats df
host_review_stats = listings_df[['host_id', 'review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
                      'review_scores_checkin','review_scores_communication','review_scores_location',
                      'review_scores_value', 'total_reviews']]
print(type(host_review_stats['host_id']))
host_review_stats

<class 'pandas.core.series.Series'>


,host_id,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,total_reviews
0,1466919,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
1,10328771,100.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0
2,19252768,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
3,10668311,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
4,24837558,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
...,...,...,...,...,...,...,...,...,...
279707,31161181,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
279708,10294858,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
279709,2238502,100.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0
279710,38633695,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0


In [15]:
#don't include rows that do not have a host_id

host_review_stats = host_review_stats.dropna(axis=0, subset=['host_id'])
#host_review_stats = host_review_stats.dropna(subset = ['host_id'])
#host_review_stats = host_review_stats[pd.notnull(host_review_stats['host_id'])]
#host_review_stats = host_review_stats.dropna(subset=['host_id'])

In [16]:
#rename columns to specify they are about the host

host_review_stats = host_review_stats.rename(columns={'review_scores_rating':'host_rating', 'review_scores_accuracy':
                                                            'host_score_accuracy', 'review_scores_checkin':'host_score_checkin',
                                                          'review_scores_communication':'host_score_communication', 
                                                      'review_scores_cleanliness': 'host_score_cleanliness',
                                                            'review_scores_location':'host_score_location','review_scores_value':'host_score_value',
                                                     'total_reviews': 'host_total_reviews'})
print(type(host_review_stats))
host_review_stats

<class 'pandas.core.frame.DataFrame'>


,host_id,host_rating,host_score_accuracy,host_score_cleanliness,host_score_checkin,host_score_communication,host_score_location,host_score_value,host_total_reviews
0,1466919,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
1,10328771,100.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0
2,19252768,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
3,10668311,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
4,24837558,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
...,...,...,...,...,...,...,...,...,...
279707,31161181,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
279708,10294858,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
279709,2238502,100.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0
279710,38633695,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0


In [17]:
#see how many host_ids there are
print(len(host_review_stats["host_id"].unique()))
#there are less than listing ids

182024


In [18]:
#try to groupby host_id and average all review scores (via .mean()),
#while summing the total reviews

host_review_stats = host_review_stats.groupby('host_id').agg({'host_rating':'mean', 'host_score_accuracy':'mean', 
                                                    'host_score_cleanliness':'mean','host_score_checkin':'mean',
                                                    'host_score_communication':'mean', 'host_score_location':'mean',
                                                    'host_score_value':'mean', 'host_total_reviews':'sum'}).round(2)

In [19]:
host_review_stats.head(10)

,host_rating,host_score_accuracy,host_score_cleanliness,host_score_checkin,host_score_communication,host_score_location,host_score_value,host_total_reviews
host_id,,,,,,,,
1822,90.00,9.00,9.00,10.00,10.00,8.00,9.00,48.0
1944,95.40,9.70,9.50,9.90,9.90,9.50,9.00,123.0
2330,69.33,7.67,7.33,9.67,9.33,8.00,7.00,13.0
2353,91.00,9.00,9.00,10.00,10.00,9.00,9.00,98.0
2438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2571,98.00,10.00,10.00,10.00,10.00,9.00,10.00,33.0
2626,97.00,10.00,8.50,10.00,10.00,10.00,10.00,272.0
2782,94.50,9.50,9.00,9.50,10.00,10.00,9.50,22.0
2787,95.71,9.57,9.57,10.00,10.00,9.57,9.43,140.0


## Move onto the host_location_review_stats dataframe

In [20]:
# begin the host_location_review_stats table
host_location_review_stats = listings_df[['host_location', 'review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
                      'review_scores_checkin','review_scores_communication','review_scores_location',
                      'review_scores_value', 'total_reviews']]
host_location_review_stats.head()

,host_location,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,total_reviews
0,"Paris, Ile-de-France, France",100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
1,"Paris, Ile-de-France, France",100.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0
2,"Paris, Ile-de-France, France",100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
3,"Paris, Ile-de-France, France",100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0
4,"Paris, Ile-de-France, France",100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0


In [21]:
#rename columns to specify they are about the host location

host_location_review_stats = host_location_review_stats.rename(columns={'review_scores_rating':'host_location_rating', 'review_scores_accuracy':
                                                            'host_location_score_accuracy', 'review_scores_checkin':'host_location_score_checkin',
                                                          'review_scores_communication':'host_location_score_communication', 
                                                      'review_scores_cleanliness': 'host_location_score_cleanliness',
                                                            'review_scores_location':'host_location_score_location','review_scores_value':'host_location_score_value',
                                                     'total_reviews': 'host_location_total_reviews'})

In [22]:
#see how many host_locations there are in the dataset
print(len(host_location_review_stats["host_location"].unique()))

7160


In [23]:
host_location_review_stats = host_location_review_stats.groupby('host_location').agg({'host_location_rating':'mean', 'host_location_score_accuracy':'mean', 
                                                    'host_location_score_cleanliness':'mean','host_location_score_checkin':'mean',
                                                    'host_location_score_communication':'mean', 'host_location_score_location':'mean',
                                                    'host_location_score_value':'mean', 'host_location_total_reviews':'sum'}).round(2)

In [24]:
host_location_review_stats.head()

,host_location_rating,host_location_score_accuracy,host_location_score_cleanliness,host_location_score_checkin,host_location_score_communication,host_location_score_location,host_location_score_value,host_location_total_reviews
host_location,,,,,,,,
"Brooklyn, NY",98.0,10.0,10.0,10.0,10.0,10.0,10.0,123.0
Australia,100.0,10.0,10.0,10.0,10.0,10.0,10.0,144.0
"Bangkok, Thailand",90.0,10.0,10.0,10.0,10.0,9.0,10.0,4.0
Brazil,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0
"Campania, Italy",87.0,9.0,9.0,10.0,10.0,9.0,9.0,36.0


## Make the reviewer_freq table and review_id table

In [41]:
reviewer_freq = reviews_df[['reviewer_id', 'review_id']]

In [42]:
reviewer_freq = reviewer_freq['reviewer_id'].value_counts()

In [43]:
reviewer_freq = reviews_df['reviewer_id'].value_counts()
reviewer_freq = pd.DataFrame(reviewer_freq)
reviewer_freq = reviewer_freq.reset_index()
reviewer_freq = reviewer_freq.rename(columns={'reviewer_id':'total_reviews'})
reviewer_freq = reviewer_freq.rename(columns={'index':'reviewer_id'})
reviewer_freq = pd.DataFrame(reviewer_freq)
reviewer_freq

,reviewer_id,total_reviews
0,43924136,283
1,49299643,96
2,45047906,83
3,179211734,82
4,188964362,71
...,...,...
4450000,126260587,1
4450001,273067372,1
4450002,21413230,1
4450003,151434607,1


In [49]:
review_id = reviews_df[['review_id', 'reviewer_id','listing_id']]
review_id

,review_id,reviewer_id,listing_id
0,330265172,11863072,11798
1,330103585,39147453,15383
2,329985788,1125378,16455
3,330016899,172717984,17919
4,329995638,17542859,26827
...,...,...,...
5373138,726766332,283094516,47779342
5373139,727963021,76411977,47823964
5373140,728548625,71370946,47896175
5373141,727399287,109011160,47900451


## Onto SQL Alchemy (after creating SQL tables in pgAdmin)

In [29]:
protocol = "postgresql"
user = "postgres"
location = "localhost"
port = "5432"
database = "etl_project_db"

connection_string = f"{protocol}://{user}:{db_password}@{location}:{port}/{database}"

In [30]:
engine = create_engine(connection_string)

In [31]:
#load data into database
listings_info.to_sql(name='listings_info', con=engine, if_exists='append', index=False)

In [32]:
listing_review_stats.to_sql(name='listing_review_stats', con=engine, if_exists='append', index=False)

In [33]:
#since host_review_stats is uses groupby, make index=True, otherwise host_id will be dropped
host_review_stats.to_sql(name='host_review_stats', con=engine, if_exists='append', index=True)

In [36]:
#since host_location_review_stats uses groupby, make index=True
host_location_review_stats.to_sql(name='host_location_review_stats', con=engine, if_exists='append',
                                 index=True)

In [45]:
reviewer_freq.to_sql(name='reviewer_freq', con=engine, if_exists="append", index=False)

In [50]:
review_id.to_sql(name='review_id', con=engine, if_exists='append', index=False)